In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 23.8 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch import optim
import optuna
from torch.utils.data import Dataset,DataLoader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [ ]:
device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:

import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fashionmnist


In [ ]:
test=pd.read_csv("/kaggle/input/fashionmnist/fashion-mnist_test.csv")
train=pd.read_csv("/kaggle/input/fashionmnist/fashion-mnist_train.csv")
train.head()


,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
x_train=train.drop("label",axis=1).values
y_train=train["label"].values
x_test=test.drop("label",axis=1).values
y_test=test["label"].values

In [ ]:
x_train=x_train/255.0
x_test=x_test/255.0


In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(60000, 784)
(10000, 784)
(60000,)
(10000,)


In [ ]:
class MyDataset(Dataset):
  def __init__(self,features,labels):
    self.features=torch.tensor(features,dtype=torch.float32).reshape(-1,1,28,28)
    self.labels=torch.tensor(labels,dtype=torch.long)
  def __len__(self):
    return len(self.features)
  def __getitem__(self,idx):
    return self.features[idx],self.labels[idx]


In [ ]:



class Model(nn.Module):
    def __init__(self, input_features):
        super(Model, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels=input_features, out_channels=32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(0.25)
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(64, 10)

        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x


In [ ]:
train_dataset=MyDataset(x_train,y_train)
test_dataset=MyDataset(x_test,y_test)

In [ ]:
train_loader=DataLoader(train_dataset,num_workers=2,batch_size=128,shuffle=True)
test_loader=DataLoader(test_dataset,num_workers=2,batch_size=128,shuffle=False)

In [ ]:
epochs=50
learning_rate=0.01
loss_fn=nn.CrossEntropyLoss()


In [ ]:
model=Model(1)
model.to(device)
optimizer=optim.SGD(lr=learning_rate,weight_decay=1e-4,params=model.parameters())
model.train()

Model(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.3, inplace=False)
    (7): Linear(in_features=64, out_features=10,

In [ ]:
for epoch in range(epochs):
  total_loss=0
  for batch_features,batch_labels in train_loader:
    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)
    output=model(batch_features)
    loss=loss_fn(output,batch_labels)
    total_loss+=loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  print(f"Epoch: {epoch+1} Loss: {total_loss/len(train_loader)}")


Epoch: 1 Loss: 1.0230811030498699
Epoch: 2 Loss: 0.6182671981071358
Epoch: 3 Loss: 0.5280242246478352
Epoch: 4 Loss: 0.4776679400696175
Epoch: 5 Loss: 0.4407269449503437
Epoch: 6 Loss: 0.41541539682254097
Epoch: 7 Loss: 0.3959286385761904
Epoch: 8 Loss: 0.3767899166482852
Epoch: 9 Loss: 0.36635744975192713
Epoch: 10 Loss: 0.3538206037618458
Epoch: 11 Loss: 0.34489763005455926
Epoch: 12 Loss: 0.33812907836965916
Epoch: 13 Loss: 0.3267235532244131
Epoch: 14 Loss: 0.3195974739439198
Epoch: 15 Loss: 0.3146900012930319
Epoch: 16 Loss: 0.3112717517085675
Epoch: 17 Loss: 0.29991597753725074
Epoch: 18 Loss: 0.3013470396876081
Epoch: 19 Loss: 0.2926049761807741
Epoch: 20 Loss: 0.2892477421173409
Epoch: 21 Loss: 0.28354828129572146
Epoch: 22 Loss: 0.278476435810264
Epoch: 23 Loss: 0.2736038273331453
Epoch: 24 Loss: 0.2752897328754732
Epoch: 25 Loss: 0.269573522211392
Epoch: 26 Loss: 0.2682515053924467
Epoch: 27 Loss: 0.2604600660550569
Epoch: 28 Loss: 0.25820220172850056
Epoch: 29 Loss: 0.257233

In [ ]:
model.eval()

Model(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Dropout2d(p=0.25, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU()
    (7): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (9): Dropout2d(p=0.25, inplace=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): ReLU()
    (6): Dropout(p=0.3, inplace=False)
    (7): Linear(in_features=64, out_features=10,

In [ ]:
with torch.no_grad():
  total=0
  correct=0
  for batch_features,batch_labels in train_loader:
    batch_features,batch_labels=batch_features.to(device),batch_labels.to(device)
    output=model(batch_features)
    _,predicted=torch.max(output,1)
    total+=batch_labels.size(0)
    correct+=(predicted==batch_labels).sum().item()
  print(f"Accuracy: {correct/total}")


Accuracy: 0.9484166666666667
